# imports

In [13]:
import pandas as pd
import re
import numpy as np
import random

In [14]:
sam_file = "E:\\Studies\\4-1\CSE 400\\codes\sam_files\\canu_05.sam"
read_file = "E:\\Studies\\4-1\\CSE 400\\drive\\inputs\\real_SAMN10819805_pacbio_00.fastq"

In [15]:
file=open(sam_file,'r')
line=file.readline()
line=line.strip()
line=line.strip('\n')
array=[]
allLineFromaln=[]
lineCountInMapFile=0
while line:
    if(line[0]=='@'):
        line=file.readline()
        line=line.strip()
        line=line.strip('\n')
        continue
    line=re.split('\t',line)
    lineCountInMapFile+=1
    temp=[]
    for j in range(12):
        temp.append(line[j])
    allLineFromaln.append(temp)
    line=file.readline()
    line=line.strip()
    line=line.strip('\n')

allLine=np.array(allLineFromaln,dtype='object')
# print(allLine.shape)
readTemplateAln=allLine[:,0]
flag=allLine[:,1]
RnameAln=allLine[:,2]
pos=allLine[:,3]
cigar=allLine[:,5]
seq=allLine[:,9]
qual=allLine[:,10]



# Hash Table

In [16]:
class HashTable:

	# Create empty bucket list of given size
	def __init__(self, size):
		self.size = size
		self.hash_table = self.create_buckets()

	def create_buckets(self):
		return [[] for _ in range(self.size)]

	# Insert values into hash map
	def set_val(self, key, val):
		
		# Get the index from the key
		# using hash function
		hashed_key = hash(key) % self.size
		
		# Get the bucket corresponding to index
		bucket = self.hash_table[hashed_key]

		found_key = False
		for index, record in enumerate(bucket):
			record_key, record_val = record
			
			# check if the bucket has same key as
			# the key to be inserted
			if record_key == key:
				found_key = True
				break

		# If the bucket has same key as the key to be inserted,
		# Update the key value
		# Otherwise append the new key-value pair to the bucket
		if found_key:
			bucket[index] = (key, val)
		else:
			bucket.append((key, val))

	# Return searched value with specific key
	def get_val(self, key):
		
		# Get the index from the key using
		# hash function
		hashed_key = hash(key) % self.size
		
		# Get the bucket corresponding to index
		bucket = self.hash_table[hashed_key]

		found_key = False
		for index, record in enumerate(bucket):
			record_key, record_val = record
			
			# check if the bucket has same key as
			# the key being searched
			if record_key == key:
				found_key = True
				break

		# If the bucket has same key as the key being searched,
		# Return the value found
		# Otherwise indicate there was no record found
		if found_key:
			return record_val
		else:
			return "No record found"

	# Remove a value with specific key
	def delete_val(self, key):
		
		# Get the index from the key using
		# hash function
		hashed_key = hash(key) % self.size
		
		# Get the bucket corresponding to index
		bucket = self.hash_table[hashed_key]

		found_key = False
		for index, record in enumerate(bucket):
			record_key, record_val = record
			
			# check if the bucket has same key as
			# the key to be deleted
			if record_key == key:
				found_key = True
				break
		if found_key:
			bucket.pop(index)
		return

	# To print the items of hash map
	def __str__(self):
		return "".join(str(item) for item in self.hash_table)


# hash_table = HashTable(50)

# # insert some values
# hash_table.set_val('gfg@example.com', 'some value')
# print(hash_table)
# print()

# hash_table.set_val('portal@example.com', 'some other value')
# print(hash_table)
# print()

# # search/access a record with key
# print(hash_table.get_val('portal@example.com'))
# print()

# # delete or remove a value
# hash_table.delete_val('portal@example.com')
# print(hash_table)


# Get Raw Read and Raw Template from pacbio Read file

In [17]:
file=open(read_file,'r')
line=file.readline()
line=line.strip()
line=line.strip('\n')
readCountRaw=0
readTemplateraw=[]
readNameraw=[]
readLengthsRaw=[]
while line:
    if line[0]=='@':
        line=re.split("@| |\t",line)
        line=line[1]
        readTemplateraw.append(line)
        read=file.readline().strip()
        read=read.strip('\n')
        readNameraw.append(read)
        readLengthsRaw.append(len(readNameraw[len(readNameraw)-1]))
        readCountRaw +=1
    line=file.readline()
file.close()



# Read distribution

In [18]:
hash_table=HashTable(readCountRaw)
for i in range(readCountRaw):
    hash_table.set_val(readTemplateraw[i],readNameraw[i])

# Generate mapped and unmapped file from aligned sam file

In [19]:

# *********************************************************
# read_template_name  flag    mapping_postion ciger   read_seq--->mapped file sequence
#@read_template_name\n read\n +read_template_name\n qual\n ---> unmapped file sequence

# ************************************************************
mappedFile=open('mappedread.txt','w')
unmappedFile=open('unmapped.txt','w')
modifiedRandomUnmapped=open('modifiedRandomSliceUnmapped.txt','w')
unmappedLengths=[]
for i in range(lineCountInMapFile):
    line=readTemplateAln[i]+'\t'+flag[i]+'\t'+RnameAln[i]+'\t'+pos[i]+'\t'+cigar[i]+'\t'+hash_table.get_val(readTemplateAln[i])+'\t'+qual[i]
    if pos[i]=='0':
        unmappedFile.write("@"+readTemplateAln[i]+"\n")
        unmappedFile.write(hash_table.get_val(readTemplateAln[i])+"\n")
        unmappedFile.write("+"+"\n")
        unmappedFile.write(qual[i]+"\n")
        unmappedLengths.append(len(hash_table.get_val(readTemplateAln[i])))

        readSequence=hash_table.get_val(readTemplateAln[i])
        qualSequence=qual[i]
        if len(readSequence) > 1000:
            space=len(readSequence)-1000
            randomSlice=random.randint(0,space)
            readSequence=readSequence[randomSlice:randomSlice+1000]
            qualSequence=qualSequence[randomSlice:randomSlice+1000]

        modifiedRandomUnmapped.write("@"+readTemplateAln[i]+"\n")
        modifiedRandomUnmapped.write(readSequence+"\n")
        modifiedRandomUnmapped.write("+"+"\n")
        modifiedRandomUnmapped.write(qualSequence+"\n")

    else:
        mappedFile.write('%s\n' % line)


mappedFile.close()
unmappedFile.close()
modifiedRandomUnmapped.close()

In [20]:

#get minimum value from unmappedLengths
min_value=min(unmappedLengths)
#get maximum value from unmappedLengths
max_value=max(unmappedLengths)
print("min ",min_value , " max ",max_value)
print("total unmapped ",len(unmappedLengths))
#position of min value
min_index=unmappedLengths.index(min_value)
print(min_index)

min  35  max  36604
total unmapped  2480
737


In [21]:
#save unmapped statistics in file
file=open('unmapped_statistics.txt','w')
file.write("TotalLength\t"+str(sum(unmappedLengths))+"\n")
file.write("Total unmapped Read Count : "+"%s" %len(unmappedLengths)+"\n")
file.write("min read Lendth "+"%s" % min_value+"\n")
file.write("max read Lendth "+"%s" %max_value+"\n")
file.close()


In [22]:
statFile=open('OverallStatistics.txt','w')
totalReadCount=readCountRaw
unmappedReadCount=(pos=='0').sum()
mappedReadCount=totalReadCount-unmappedReadCount
totalReadCount="%s" % totalReadCount
unmappedReadCount="%s" % unmappedReadCount
mappedReadCount="%s" % mappedReadCount
line="Total Read : "+totalReadCount+"\t"+"Mapped Read Count : "+mappedReadCount+"\t"+"Unmapped Read Count : "+unmappedReadCount
statFile.write('%s' %line)
statFile.close()

# unmapped statistics

In [ ]:
statFile=open('stat.txt','w')
totalReadCount=readCountRaw
unmappedReadCount=(pos=='0').sum()
mappedReadCount=totalReadCount-unmappedReadCount
totalReadCount="%s" % totalReadCount
unmappedReadCount="%s" % unmappedReadCount
line=totalReadCount+"\t"+unmappedReadCount+"\t"+"%s" %max_value
statFile.write('%s' %line)
statFile.close()